<a href="https://colab.research.google.com/github/PassionateAbdullah/Pytorch/blob/main/Introduction_to_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# nn module from PyTorch. The nn module provides classes and functions for building neural networks
#torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels
#and DataLoader wraps an iterable around the Dataset.

#PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets.

#For this tutorial, we will be using a TorchVision dataset.

#(This imports the ToTensor transformation from Torchvision,
#This transformation converts images (usually in PIL Image format) into PyTorch tensors and normalizestheir pixel values
#to the range [0, 1]. It is often used as a preprocessing step before feeding data into a neural network.)

In [27]:
import torch
from torch import nn
from torch. utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

**Working with Data**

In [28]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [29]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader: # loop iterates over the batches in test_dataloader, x(input data, typically images),y(output data)
    print(f"Shape of X [N, C, H, W]: {X.shape}") # N(batch size), C(Number of channel eg : 1 for MNIST, 3 for RGB), H,w(height and width of image)
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [30]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


**Creating Models**

In [45]:
# Define model
class NeuralNetwork(nn.Module): #nn.module is base class of PyTorch, providing essential functionalities like parameter management and forward pass definition.
    def __init__(self): #The constructor initializes the components of the neural network.
        super().__init__() #Essential for setting up PyTorch’s internal structures for handling layers and parameters
        self.flatten = nn.Flatten() #A layer that reshapes the input tensor into a 1D vector
        self.linear_relu_stack = nn.Sequential(  #Combines multiple layers in a specified sequence.
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


**Optimizing the Model Parameters**

In [46]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

**Summary of Function Behavior:**

* Loops through all the batches of data in the dataloader.   

*   Moves data to the appropriate device (CPU/GPU).

*  Computes the predictions, loss, and gradients using backpropagation.
*   Updates the model parameters using the optimizer.


*   
Resets gradients for the next batch.
Logs progress every 100 batches



In [47]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        loss.backward()  #(calculate loss of the gradients using backpropagation)
        optimizer.step() #update the model parameter (weight)  using the calculated gradients based on the optimization algorithm (e.g., SGD or Adam).
        optimizer.zero_grad() #Resets the gradients to zero to prevent accumulation from previous batches

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [48]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [49]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.317433  [   64/60000]
loss: 2.301911  [ 6464/60000]
loss: 2.276635  [12864/60000]
loss: 2.258814  [19264/60000]
loss: 2.263512  [25664/60000]
loss: 2.217347  [32064/60000]
loss: 2.226463  [38464/60000]
loss: 2.197318  [44864/60000]
loss: 2.201909  [51264/60000]
loss: 2.146499  [57664/60000]
Test Error: 
 Accuracy: 40.0%, Avg loss: 2.153066 

Epoch 2
-------------------------------
loss: 2.174035  [   64/60000]
loss: 2.159921  [ 6464/60000]
loss: 2.098202  [12864/60000]
loss: 2.108566  [19264/60000]
loss: 2.064562  [25664/60000]
loss: 1.999920  [32064/60000]
loss: 2.024896  [38464/60000]
loss: 1.948643  [44864/60000]
loss: 1.964404  [51264/60000]
loss: 1.870773  [57664/60000]
Test Error: 
 Accuracy: 57.5%, Avg loss: 1.878069 

Epoch 3
-------------------------------
loss: 1.915822  [   64/60000]
loss: 1.884043  [ 6464/60000]
loss: 1.763600  [12864/60000]
loss: 1.804558  [19264/60000]
loss: 1.693933  [25664/60000]
loss: 1.644192  [32064/600

**Saving Model**

In [50]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


**Loading Model**

In [51]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [52]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
